In [12]:
import joblib
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors

Import the data that I had done preprocessing

In [4]:
tmp = []
for i in range(5):
    tt= joblib.load('./fea_sel/text/clean-part-{}.pkl'.format(i))
    tmp .append(tt)
X_train = pd.concat(tmp)

tmp = []
for i in range(3):
    tt= joblib.load('./fea_sel/test/clean-part-{}.pkl'.format(i))
    tmp .append(tt)
X_test = pd.concat(tmp)

###### Check the data 

In [6]:
X_train.head()

,_score,_index,_crawldate,_type,hashtags,tweet_id,text,clean_sent
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...","[people, who, post, add, <allcaps>, pron, </al..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[<user>, as, <allcaps>, pron, </allcaps>, -, s..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,"[now, <allcaps>, issa, </allcaps>, be, stalk, ..."
5,120,hashtag_tweets,2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[<user>, <user>, thx, for, the, <allcaps>, bes..."
6,1021,hashtag_tweets,2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,"[still, wait, on, those, supply, liscus]"


In [7]:
X_test.head()

,_score,_index,_crawldate,_type,hashtags,tweet_id,text,clean_sent
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...","[confident, of, <allcaps>, pron, </allcaps>, -..."
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s...","[trust, be, not, the, same, as, faith, a, frie..."
9,66,hashtag_tweets,2015-09-09 09:22:55,tweets,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,"[when, do, <allcaps>, pron, </allcaps>, -, hav..."
30,104,hashtag_tweets,2015-10-10 14:33:26,tweets,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...","[god, wake, <allcaps>, pron, </allcaps>, -, up..."
33,310,hashtag_tweets,2016-10-23 08:49:50,tweets,[],0x26289a,"In these tough times, who do YOU turn to as yo...","[in, these, tough, time, who, do, you, turn, t..."


##### Train my word2vec model
This will take some time...

In [14]:
#X_train['token']=X_train['clean_sent'].apply(eval)
sentences = X_train['clean_sent']

tweet_model_dim = 100
tweet_model = Word2Vec(sentences, 
                       size=tweet_model_dim,
                       sg=1,
                       hs=1,
                       window=5,
                       negative=10,
                       min_count=10,
                       iter=20,
                       workers=8)

#joblib.dump(tweet_model,'./fea_sel/embed_model/tweet_model.pkl')

In [15]:
tweet_model

In [16]:
tweet_model.wv.vocab.items()

dict_items([('people', <gensim.models.keyedvectors.Vocab object at 0x000001BF974B4A48>), ('who', <gensim.models.keyedvectors.Vocab object at 0x000001BF974B3688>), ('post', <gensim.models.keyedvectors.Vocab object at 0x000001BF974B3108>), ('add', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CD48>), ('<allcaps>', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CA88>), ('pron', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CD88>), ('</allcaps>', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CDC8>), ('-', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CE08>), ('on', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CC48>), ('snapchat', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CD08>), ('"', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CE48>), ('must', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CE88>), ('be', <gensim.models.keyedvectors.Vocab object at 0x000001BE8BE3CEC8>), ('dehydr

###### prepare the word2id for latter "text_to_sequence" and embedding for the model

In [17]:
#tweet_model = joblib.load('./fea_sel/embed_model/tweet_model.pkl')
tweet_model_word2id = {w:v.index+1 for w, v in tweet_model.wv.vocab.items()}
tweet_model_embedding = np.vstack([np.zeros(tweet_model_dim, dtype='float32'), tweet_model.wv.vectors])
#joblib.dump(tweet_model_embedding,'./fea_sel/embed_model/tweet_model_embedding.pkl')

In [18]:
def text_to_sequence(text, word2id):
    seq = []
    for w in text:
        i = word2id.get(w)
        if i is not None:
            seq.append(i) 
        else:
            seq.append(0) 
    return seq

In [19]:
X_train['seq_tweet_embedd'] = X_train['clean_sent'].apply(text_to_sequence, args=(tweet_model_word2id, ))
X_test['seq_tweet_embedd'] = X_test['clean_sent'].apply(text_to_sequence, args=(tweet_model_word2id, ))
tweet_embedd_max_length = 100
X_train_2 = pad_sequences(X_train['seq_tweet_embedd'], maxlen=tweet_embedd_max_length)
X_test_2 = pad_sequences(X_test['seq_tweet_embedd'], maxlen=tweet_embedd_max_length)
#joblib.dump(X_test,'./fea_sel/embed_model/X_test.pkl')

In [23]:
X_train['seq_tweet_embedd']

0          [67, 64, 374, 658, 2, 4, 3, 1, 21, 1109, 36, 2...
1          [7, 63, 2, 4, 3, 1, 61, 198, 5, 2264, 9, 22662...
3                 [69, 2, 557, 3, 5, 3745, 2182, 76, 76, 76]
5          [7, 7, 1576, 13, 8, 2, 756, 3, 60, 172, 35, 42...
6                               [106, 127, 21, 183, 2653, 0]
                                 ...                        
1867526    [2, 4, 3, 1, 5, 25, 98, 18, 18, 0, 8, 258, 16,...
1867527    [15, 153, 0, 2, 4, 3, 1, 87, 48, 9, 5, 247, 9,...
1867528    [81, 5, 1173, 244, 192, 319, 276, 8, 2710, 31,...
1867533    [1054, 3895, 33, 23, 158, 2, 4, 3, 1, 32, 594,...
1867534                              [138, 9, 5, 140, 13322]
Name: seq_tweet_embedd, Length: 1455563, dtype: object

In [20]:
X_train_2

array([[    0,     0,     0, ...,    14,    22,     5],
       [    0,     0,     0, ...,     2,  1662,     3],
       [    0,     0,     0, ...,    76,    76,    76],
       ...,
       [    0,     0,     0, ...,   786,  3555, 28024],
       [    0,     0,     0, ...,     4,     3,     1],
       [    0,     0,     0, ...,     5,   140, 13322]])